## Imports

In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

from modulos.funcoes_sanity_check import categorias_colunas

pd.set_option('display.max_columns', None)

## Base de dados - Tratamento e Sanity Check

In [7]:
us_states = {
    "AL": "Alabama",
    "AK": "Alaska",
    "AZ": "Arizona",
    "AR": "Arkansas",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "FL": "Florida",
    "GA": "Georgia",
    "HI": "Hawaii",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "IA": "Iowa",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "ME": "Maine",
    "MD": "Maryland",
    "MA": "Massachusetts",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MS": "Mississippi",
    "MO": "Missouri",
    "MT": "Montana",
    "NE": "Nebraska",
    "NV": "Nevada",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NY": "New York",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PA": "Pennsylvania",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VT": "Vermont",
    "VA": "Virginia",
    "WA": "Washington",
    "WV": "West Virginia",
    "WI": "Wisconsin",
    "WY": "Wyoming"
}

In [2]:
def faixa_dia_mes(x):
    if x <= 5:
        return "Até dia 5"
    if x <= 10:
        return "De 6 a 10"
    if x <= 15:
        return "De 11 a 15"
    if x <= 20:
        return "De 16 a 20"
    if x <= 25:
        return "De 21 a 25"
    else:
        return "Depois do dia 26"

In [3]:
df_pizza = pd.read_excel('dados/Enhanced_pizza_sell_data_2024-25.xlsx')
df_pizza['Restaurant Name'] = np.where(df_pizza['Restaurant Name'] == 'Marco’s Pizza', "Marco's Pizza", df_pizza['Restaurant Name'])
df_pizza = df_pizza.drop(['Delivery Time', 'Topping Density', 'Order Month', 'Payment Category', 'Pizza Complexity', 'Traffic Impact', 'Restaurant Avg Time', 
                          'Is Weekend', 'Toppings Count'], axis = 1)
df_pizza['Month'] = df_pizza['Order Time'].astype(str).apply(lambda x: x[:7])
df_pizza['Month Day'] = df_pizza['Order Time'].dt.day.apply(faixa_dia_mes)
df_pizza['Weekday'] = df_pizza['Order Time'].dt.day_name()
df_pizza['Order Hour'] = df_pizza['Order Time'].astype(str).apply(lambda x: x[11:16])
df_pizza['State'] = df_pizza['Location'].apply(lambda x: x.split(', ')[1])
print(df_pizza.shape)
df_pizza.head()

(1004, 20)


,Order ID,Restaurant Name,Location,Order Time,Delivery Duration (min),Pizza Size,Pizza Type,Distance (km),Traffic Level,Payment Method,Is Peak Hour,Delivery Efficiency (min/km),Estimated Duration (min),Delay (min),Is Delayed,Order Hour,Month,Month Day,Weekday,State
0,ORD001,Domino's,"New York, NY",2024-01-05 18:30:00,15,Medium,Veg,2.5,Medium,Card,True,6.000000,6.0,9.0,False,18:30,2024-01,Até dia 5,Friday,NY
1,ORD002,Papa John's,"Los Angeles, CA",2024-02-14 20:00:00,25,Large,Non-Veg,5.0,High,Wallet,True,5.000000,12.0,13.0,False,20:00,2024-02,De 11 a 15,Wednesday,CA
2,ORD003,Little Caesars,"Chicago, IL",2024-03-21 12:15:00,20,Small,Vegan,3.0,Low,UPI,False,6.666667,7.2,12.8,False,12:15,2024-03,De 21 a 25,Thursday,IL
3,ORD004,Pizza Hut,"Miami, FL",2024-04-10 19:45:00,25,XL,Cheese Burst,4.5,Medium,Cash,True,5.555556,10.8,14.2,False,19:45,2024-04,De 6 a 10,Wednesday,FL
4,ORD005,Marco's Pizza,"Dallas, TX",2024-05-05 13:00:00,20,Medium,Non-Veg,2.0,High,Card,False,10.000000,4.8,15.2,False,13:00,2024-05,Até dia 5,Sunday,TX


In [106]:
categorias_colunas(df = df_pizza, n_max = 12)

Coluna: Restaurant Name | Qtd categorias: 5 | Qtd missings: 0 (0.0%)
Restaurant Name
Domino's          212
Papa John's       204
Little Caesars    199
Marco's Pizza     195
Pizza Hut         194
Name: count, dtype: int64
--------------------------------------------------

Coluna: Delivery Duration (min) | Qtd categorias: 8 | Qtd missings: 0 (0.0%)
Delivery Duration (min)
30    437
20    233
25    123
40     92
35     44
50     40
45     34
15      1
Name: count, dtype: int64
--------------------------------------------------

Coluna: Pizza Size | Qtd categorias: 4 | Qtd missings: 0 (0.0%)
Pizza Size
Medium    429
Large     240
XL        203
Small     132
Name: count, dtype: int64
--------------------------------------------------

Coluna: Pizza Type | Qtd categorias: 12 | Qtd missings: 0 (0.0%)
Pizza Type
Non-Veg          216
Veg              202
Cheese Burst     188
Vegan            132
Sicilian          64
BBQ Chicken       54
Margarita         54
Gluten-Free       41
Thai Chicken   

## Gráficos intermediários

In [ ]:
## Gráfico


# dropdown: restaurante
# gráfico choropleth: qtd de pedidos por estado por mês (slider)
# linhas: média eficiência de delivery por hora:min + média de delay por hora:min
# bolhas: distancia vs tempo de delivery

### Mapa

In [102]:
def grafico_mapa(us_states, restaurante = None):

    if restaurante == None:
        df = df_pizza.copy()
        restaurante_name = 'Todos os restaurantes'
    else:
        df = df_pizza[df_pizza['Restaurant Name'] == restaurante]
        restaurante_name = restaurante

    df = df[['State', 'Month']].value_counts().reset_index()
    df.columns = ['State', 'Month', 'count']
    df['Name'] = df['State'].map(us_states)
    df['Name'] = df['Name'] + " (" + df['State'] + ")"

    # Certifique-se de que Month seja datetime (se ainda não for)
    df['Month'] = pd.to_datetime(df['Month'])

    # Criar uma coluna formatada
    df['Month_str'] = df['Month'].dt.strftime('%b/%Y')  # 'jan/2024', 'fev/2024', etc.

    # Obter meses únicos formatados, mantendo a ordem cronológica
    meses_ordenados = df.sort_values('Month')['Month_str'].unique()

    # Criar os traces e slider
    data = []
    steps = []

    for i, mes_str in enumerate(meses_ordenados):
        df_mes = df[df['Month_str'] == mes_str]
        
        trace = go.Choropleth(
            locations=df_mes['State'],
            z=df_mes['count'],
            locationmode='USA-states',
            colorscale='Reds',
            zmin=df['count'].min(),
            zmax=df['count'].max(),
            customdata=df_mes['Name'],
            hovertemplate='%{customdata}<br>%{z}<extra></extra>',
            visible=(i == 0)
        )
        data.append(trace)
        
        step = dict(
            method="update",
            args=[{"visible": [j == i for j in range(len(meses_ordenados))]},
                {"title": f"Quantidade de pedidos por Estado | {restaurante_name} | {mes_str}"}],
            label=mes_str
        )
        steps.append(step)

    sliders = [dict(
        active=0,
        steps=steps,
        currentvalue={"prefix": "Mês: "}
    )]

    fig = go.Figure(data=data)
    # fig.update_layout(
    #     title_text=f"Quantidade de pedidos por Estado | {restaurante_name}",
    #     geo_scope='usa',
    #     sliders=sliders,
    #     height = 700,
    #     width = 900
    # )

    # fig.show()
    return fig

# grafico_mapa(us_states, restaurante = "Domino's")


### Gráfico de linhas eficiência + delay

In [99]:
def grafico_linhas(restaurante):

    df_tmp = df_pizza[df_pizza['Restaurant Name'] == restaurante]
    qtd_pedidos = df_tmp[['Order Hour']].value_counts().reset_index(name='value').assign(variable='Qtd de pedidos')
    eficiencia = df_tmp.groupby(['Order Hour'])['Delivery Efficiency (min/km)'].mean().reset_index(name='value').assign(variable='Delivery Efficiency (min/km)')
    delay = df_tmp.groupby(['Order Hour'])['Delay (min)'].mean().reset_index(name='value').assign(variable='Delay (min)')
    df_plot_raw = pd.concat([eficiencia, delay, qtd_pedidos])

    # Obter todos os valores únicos
    # all_restaurants = df_plot_raw['Restaurant Name'].unique()
    all_hours = sorted(df_tmp['Order Hour'].unique())
    all_variables = df_plot_raw['variable'].unique()

    # Criar produto cartesiano
    full_grid = (
        pd.MultiIndex.from_product(
            [all_hours, all_variables],
            names=['Order Hour', 'variable']
        )
        .to_frame(index=False)
    )

    df_plot = full_grid.merge(df_plot_raw, on=['Order Hour', 'variable'], how='left')

    # Garantir que Order Hour é categórico ordenado
    df_plot['Order Hour'] = pd.Categorical(df_plot['Order Hour'], categories=all_hours, ordered=True)

    # Ordenar corretamente
    df_plot = df_plot.sort_values(['variable', 'Order Hour'])

    # Criar o gráfico
    fig = px.line(
        df_plot,
        x='Order Hour',
        y='value',
        markers=True,
        color = 'variable'
    )

    fig.update_yaxes(range=[0, 60], title = '', showgrid=True, gridcolor='lightgray')
    fig.update_xaxes(title = '', showgrid=True, gridcolor='lightgray')

    fig.update_traces(connectgaps=True)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

    fig.update_layout(
        legend=dict(orientation="h",
                    yanchor="bottom",
                    y=-0.5,
                    xanchor="center",
                    x=0.5,
                    title = ''),
        height = 600,
        plot_bgcolor = 'white',
        title = f"Eficiência de delivery, Delay e Quantidade de pedidos | {restaurante}")

    fig.show()

grafico_linhas(restaurante = "Domino's")

### Gráfico de bolhas duração do delivery x distância

In [100]:
def grafico_bolhas(restaurante):
    df_tmp = df_pizza[df_pizza['Restaurant Name'] == restaurante]
    fig = px.scatter(df_tmp[['Distance (km)', 'Delivery Duration (min)']].value_counts().reset_index(),
            x = 'Distance (km)', y = 'Delivery Duration (min)', 
            size = 'count')
    fig.update_yaxes(range=[0, 60], showgrid=True, gridcolor='lightgray')
    fig.update_xaxes(showgrid=True, gridcolor='lightgray')
    fig.update_layout(plot_bgcolor = 'white', height = 600, title = f"Duração de delivery vs. Distância | {restaurante}")
    fig.show()

grafico_bolhas(restaurante = "Domino's")

## Gráfico com dropdown

In [105]:
fig = make_subplots(rows = 2, cols = 2,
                    specs=[[{"rowspan":2}, {}],
                           [None, {}]])

def add_traces(fig, restaurante):

       # if restaurante == "Domino's":
       #        visible = True
       # else:
       #        visible = False

       # Gráfico de mapa -------------------------------------------------------------------------------------------------
       df = df_pizza[df_pizza['Restaurant Name'] == restaurante]

       df = df[['State', 'Month']].value_counts().reset_index()
       df.columns = ['State', 'Month', 'count']
       df['Name'] = df['State'].map(us_states)
       df['Name'] = df['Name'] + " (" + df['State'] + ")"
       df['Month'] = pd.to_datetime(df['Month'])
       df['Month_str'] = df['Month'].dt.strftime('%b/%Y')

       meses_ordenados = df.sort_values('Month')['Month_str'].unique()

       data = []
       steps = []

       for i, mes_str in enumerate(meses_ordenados):
              df_mes = df[df['Month_str'] == mes_str]
              
              trace = go.Choropleth(
              locations=df_mes['State'],
              z=df_mes['count'],
              locationmode='USA-states',
              colorscale='Reds',
              zmin=df['count'].min(),
              zmax=df['count'].max(),
              customdata=df_mes['Name'],
              hovertemplate='%{customdata}<br>%{z}<extra></extra>',
              visible=(i == 0)
              )
              data.append(trace)
              
              step = dict(
              method="update",
              args=[{"visible": [j == i for j in range(len(meses_ordenados))]},
                     {"title": f"Quantidade de pedidos por Estado | {restaurante} | {mes_str}"}],
              label=mes_str
              )
              steps.append(step)

       sliders = [dict(
              active=0,
              steps=steps,
              currentvalue={"prefix": "Mês: "}
       )]

       fig.add_trace(go.Figure(data=data), row = 1, col = 1)
       fig.update_layout(
              title_text=f"Quantidade de pedidos por Estado | {restaurante}",
              geo_scope='usa',
              sliders=sliders
       )

       # Gráfico de linhas -----------------------------------------------------------------------------------------------

       df_tmp = df_pizza[df_pizza['Restaurant Name'] == restaurante]
       qtd_pedidos = df_tmp[['Order Hour']].value_counts().reset_index(name='value').assign(variable='Qtd de pedidos')
       eficiencia = df_tmp.groupby(['Order Hour'])['Delivery Efficiency (min/km)'].mean().reset_index(name='value').assign(variable='Delivery Efficiency (min/km)')
       delay = df_tmp.groupby(['Order Hour'])['Delay (min)'].mean().reset_index(name='value').assign(variable='Delay (min)')
       df_plot_raw = pd.concat([eficiencia, delay, qtd_pedidos])

       # Obter todos os valores únicos
       # all_restaurants = df_plot_raw['Restaurant Name'].unique()
       all_hours = sorted(df_tmp['Order Hour'].unique())
       all_variables = df_plot_raw['variable'].unique()

       # Criar produto cartesiano
       full_grid = (
              pd.MultiIndex.from_product(
              [all_hours, all_variables],
              names=['Order Hour', 'variable']
              )
              .to_frame(index=False)
       )

       df_plot = full_grid.merge(df_plot_raw, on=['Order Hour', 'variable'], how='left')

       # Garantir que Order Hour é categórico ordenado
       df_plot['Order Hour'] = pd.Categorical(df_plot['Order Hour'], categories=all_hours, ordered=True)

       # Ordenar corretamente
       df_plot = df_plot.sort_values(['variable', 'Order Hour'])

       # Criar o gráfico
       fig = px.line(
              df_plot,
              x='Order Hour',
              y='value',
              markers=True,
              color = 'variable'
       )

       fig.update_yaxes(range=[0, 60], title = '', showgrid=True, gridcolor='lightgray')
       fig.update_xaxes(title = '', showgrid=True, gridcolor='lightgray')

       fig.update_traces(connectgaps=True)
       fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

       fig.update_layout(
              legend=dict(orientation="h",
                     yanchor="bottom",
                     y=-0.5,
                     xanchor="center",
                     x=0.5,
                     title = ''),
              height = 600,
              plot_bgcolor = 'white',
              title = f"Eficiência de delivery, Delay e Quantidade de pedidos | {restaurante}")



       # Gráfico de bolhas -----------------------------------------------------------------------------------------------




add_traces(fig = fig, restaurante = "Domino's")

fig.show()

ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: [Figure({
    'data': [{'colorscale': [[0.0, 'rgb(255,245,240)'], [0.125,
                             'rgb(254,224,210)'], [0.25, 'rgb(252,187,161)'],
                             [0.375, 'rgb(252,146,114)'], [0.5, 'rgb(251,106,74)'],
                             [0.625, 'rgb(239,59,44)'], [0.75, 'rgb(203,24,29)'],
                             [0.875, 'rgb(165,15,21)'], [1.0, 'rgb(103,0,13)']],
              'customdata': array(['New York (NY)', 'Washington (WA)', 'Florida (FL)', 'Colorado (CO)',
                                   'California (CA)'], dtype=object),
              'hovertemplate': '%{customdata}<br>%{z}<extra></extra>',
              'locationmode': 'USA-states',
              'locations': array(['NY', 'WA', 'FL', 'CO', 'CA'], dtype=object),
              'type': 'choropleth',
              'visible': True,
              'z': array([2, 1, 1, 1, 1], dtype=int64),
  

## [Exemplo original]

In [144]:
# fig = make_subplots(rows = 1, cols = 5, shared_yaxes=True)

# def add_traces(fig, df_vol, delta_jan_fev, delta_fev_mar, visible):
   
#     # Gráficos de volume de cada mês ----------------------------------------------------------------
#     for i in [1, 2, 3]:
#         fig.add_trace(
#             go.Bar(x = df_vol[df_vol['DAT_REF'] == f'2025-0{str(i)}']['qtd'],
#                    y = df_vol[df_vol['DAT_REF'] == f'2025-0{str(i)}']['TIPO'],
#                    text=df_vol[df_vol['DAT_REF'] == f'2025-0{str(i)}']['qtd_pct'],
#                    customdata = df_vol[df_vol['DAT_REF'] == f'2025-0{str(i)}']['DAT_REF'],
#                    hovertemplate = "<b>%{customdata}</b> <br><b>Motivo:</b> %{y} <br><b>Quantidade:</b> %{text}<extra></extra>",
#                    textposition='outside',
#                    textfont=dict(size=10),
#                    orientation = 'h', showlegend=False, width=0.7, marker_color = 'gray', visible = visible),
#             row = 1,
#             col = i
#         )
   
#     # Gráficos de delta ------------------------------------------------------------------------------
#     for idx, df in enumerate([delta_jan_fev, delta_fev_mar]):
#         fig.add_trace(
#             go.Bar(x = df['pct'],
#                    y = df['TIPO'],
#                    text= [f"{v:.1%}" for v in df['pct']],
#                    textposition='outside',
#                    textfont=dict(size=10),
#                    customdata = df['delta'],
#                    hovertemplate = "<b>Motivo:</b> %{y} <br><b>Δ Volume:</b> %{customdata} (%{text})<extra></extra>",
#                    orientation = 'h', showlegend=False, width=0.7,
#                    marker_color = ['#ac3a4e' if val > 0 else 'darkgreen' for val in df['delta']],
#                    visible = visible),
#             row = 1,
#             col = idx + 4
#         )
       
# # Traces - Todas as Plataformas
# add_traces(fig = fig,
#            df_vol = comp_mes_2025,
#            delta_jan_fev = delta_jan_fev,
#            delta_fev_mar = delta_fev_mar,
#            visible = True)

# # Traces - Controle
# add_traces(fig = fig,
#            df_vol = comp_mes_2025_plataformas[comp_mes_2025_plataformas['PRODUTO_CONSOLIDADO_2'] == 'Controle'],
#            delta_jan_fev = delta_jan_fev_c,
#            delta_fev_mar = delta_fev_mar_c,
#            visible = False)

# # Traces - Pós-pago
# add_traces(fig = fig,
#            df_vol = comp_mes_2025_plataformas[comp_mes_2025_plataformas['PRODUTO_CONSOLIDADO_2'] == 'Pós-pago'],
#            delta_jan_fev = delta_jan_fev_p,
#            delta_fev_mar = delta_fev_mar_p,
#            visible = False)

# title = dict(
#     text="<b>[Comparação mensal] Canais Críticos</b> | Todas as Causas N1",
#     y=0.95,
#     yanchor='top'
# )

# # volumes totais
# vol_jan_c = str(sep_milhar(comp_mes_2025_plataformas[(comp_mes_2025_plataformas['PRODUTO_CONSOLIDADO_2'] == 'Controle') & (comp_mes_2025_plataformas['DAT_REF'] == '2025-01')]['qtd'].sum()))
# vol_fev_c = str(sep_milhar(comp_mes_2025_plataformas[(comp_mes_2025_plataformas['PRODUTO_CONSOLIDADO_2'] == 'Controle') & (comp_mes_2025_plataformas['DAT_REF'] == '2025-02')]['qtd'].sum()))
# vol_mar_c = str(sep_milhar(comp_mes_2025_plataformas[(comp_mes_2025_plataformas['PRODUTO_CONSOLIDADO_2'] == 'Controle') & (comp_mes_2025_plataformas['DAT_REF'] == '2025-03')]['qtd'].sum()))

# vol_jan_p = str(sep_milhar(comp_mes_2025_plataformas[(comp_mes_2025_plataformas['PRODUTO_CONSOLIDADO_2'] == 'Pós-pago') & (comp_mes_2025_plataformas['DAT_REF'] == '2025-01')]['qtd'].sum()))
# vol_fev_p = str(sep_milhar(comp_mes_2025_plataformas[(comp_mes_2025_plataformas['PRODUTO_CONSOLIDADO_2'] == 'Pós-pago') & (comp_mes_2025_plataformas['DAT_REF'] == '2025-02')]['qtd'].sum()))
# vol_mar_p = str(sep_milhar(comp_mes_2025_plataformas[(comp_mes_2025_plataformas['PRODUTO_CONSOLIDADO_2'] == 'Pós-pago') & (comp_mes_2025_plataformas['DAT_REF'] == '2025-03')]['qtd'].sum()))

# lista_x = [0.02, 0.23, 0.5, 0.71, 0.92]
# altura_y = 1.07

# def get_annotations(option):
#     if option == "Controle + Pós-pago":
#         return [
#             dict(text=f"<b>Jan/2025</b><br>Volume total: {str(sep_milhar(comp_mes_2025[comp_mes_2025['DAT_REF'] == '2025-01']['qtd'].sum()))}",
#                  x=lista_x[0], y=altura_y, font = dict(size=12, color='black'), xref = 'paper', yref = 'paper', showarrow = False),
#             dict(text=f"<b>Fev/2025</b><br>Volume total: {str(sep_milhar(comp_mes_2025[comp_mes_2025['DAT_REF'] == '2025-02']['qtd'].sum()))}",
#                  x=lista_x[1], y=altura_y, font = dict(size=12, color='black'), xref = 'paper', yref = 'paper', showarrow = False),
#             dict(text=f"<b>Mar/2025</b><br>Volume total: {str(sep_milhar(comp_mes_2025[comp_mes_2025['DAT_REF'] == '2025-03']['qtd'].sum()))}",
#                  x=lista_x[2], y=altura_y, font = dict(size=12, color='black'), xref = 'paper', yref = 'paper', showarrow = False),
#             dict(text=f"<b>Δ Jan → Fev</b><br>Δ Volume: {str(sep_milhar(vol_delta_jan_fev))}",
#                  x=lista_x[3], y=altura_y, font = dict(size=12, color='black'), xref = 'paper', yref = 'paper', showarrow = False),
#             dict(text=f"<b>Δ Fev → Mar</b><br>Δ Volume: {str(sep_milhar(vol_delta_fev_mar))}",
#                  x=lista_x[4], y=altura_y, font = dict(size=12, color='black'), xref = 'paper', yref = 'paper', showarrow = False)
#         ]
#     elif option == "Controle":
#         return [
#             dict(text=f"<b>Jan/2025</b><br>Volume total: {vol_jan_c}",
#                  x=lista_x[0], y=altura_y, font = dict(size=12, color='black'), xref = 'paper', yref = 'paper', showarrow = False),
#             dict(text=f"<b>Fev/2025</b><br>Volume total: {vol_fev_c}",
#                  x=lista_x[1], y=altura_y, font = dict(size=12, color='black'), xref = 'paper', yref = 'paper', showarrow = False),
#             dict(text=f"<b>Mar/2025</b><br>Volume total: {vol_mar_c}",
#                  x=lista_x[2], y=altura_y, font = dict(size=12, color='black'), xref = 'paper', yref = 'paper', showarrow = False),
#             dict(text=f"<b>Δ Jan → Fev</b><br>Δ Volume: {str(sep_milhar(vol_delta_jan_fev_c))}",
#                  x=lista_x[3], y=altura_y, font = dict(size=12, color='black'), xref = 'paper', yref = 'paper', showarrow = False),
#             dict(text=f"<b>Δ Fev → Mar</b><br>Δ Volume: {str(sep_milhar(vol_delta_fev_mar_c))}",
#                  x=lista_x[4], y=altura_y, font = dict(size=12, color='black'), xref = 'paper', yref = 'paper', showarrow = False)
#         ]
#     elif option == "Pós-pago":
#         return [
#             dict(text=f"<b>Jan/2025</b><br>Volume total: {vol_jan_p}",
#                  x=lista_x[0], y=altura_y, font = dict(size=12, color='black'), xref = 'paper', yref = 'paper', showarrow = False),
#             dict(text=f"<b>Fev/2025</b><br>Volume total: {vol_fev_p}",
#                  x=lista_x[1], y=altura_y, font = dict(size=12, color='black'), xref = 'paper', yref = 'paper', showarrow = False),
#             dict(text=f"<b>Mar/2025</b><br>Volume total: {vol_mar_p}",
#                  x=lista_x[2], y=altura_y, font = dict(size=12, color='black'), xref = 'paper', yref = 'paper', showarrow = False),
#             dict(text=f"<b>Δ Jan → Fev</b><br>Δ Volume: {str(sep_milhar(vol_delta_jan_fev_p))}",
#                  x=lista_x[3], y=altura_y, font = dict(size=12, color='black'), xref = 'paper', yref = 'paper', showarrow = False),
#             dict(text=f"<b>Δ Fev → Mar</b><br>Δ Volume: {str(sep_milhar(vol_delta_fev_mar_p))}",
#                  x=lista_x[4], y=altura_y, font = dict(size=12, color='black'), xref = 'paper', yref = 'paper', showarrow = False)
#         ]

# buttons = [
#     dict(label="Controle + Pós-pago",
#          method="update",
#          args=[{"visible": [True] * 5 + [False] * 5 + [False] * 5},
#                {"title": title,
#                 "annotations": get_annotations("Controle + Pós-pago")}]),
#     dict(label="Controle",
#          method="update",
#          args=[{"visible": [False] * 5 + [True] * 5 + [False] * 5},
#                {"title": title,
#                 "annotations": get_annotations("Controle")}]),
#     dict(label="Pós-pago",
#          method="update",
#          args=[{"visible": [False] * 5 + [False] * 5 + [True] * 5},
#                {"title": title,
#                 "annotations": get_annotations("Pós-pago")}])
# ]
   
# fig.update_layout(height=800, width=1500, plot_bgcolor='white', paper_bgcolor='white',
#                   margin=dict(t=150, b=50, l=50, r=50),
#                   updatemenus=[dict(active=0, buttons=buttons, x=-0.087, y=1.15, xanchor='left', yanchor='top',
#                                     pad={"r": 10, "t": 10},
#                                     font=dict(size=12),
#                                     bgcolor='#e4e4e4',
#                                     bordercolor='gray',
#                                     borderwidth=1.5)],
#                   annotations=get_annotations("Controle + Pós-pago"),
#                   title = title,
#                   hoverlabel=dict(bgcolor='white'),
#                   font=dict(size=12))

# for i in [1, 2, 3]:
#     fig.update_xaxes(row = 1, col = i, title_font = {"size": 12}, tickfont = dict(size = 10, color = 'lightgray'), range=[0, 4000], dtick=1000)
#     fig.add_shape(row = 1, col = i, type="line", x0=0, x1=0, y0=-1, y1=20, yref='paper',line=dict(color="Black", width=1, dash="solid"))
   
# for i in [4, 5]:
#     fig.update_xaxes(row = 1, col = i, title_font = {"size": 12}, tickfont = dict(size = 10, color = 'lightgray'), range=[-5, 12], dtick=5, tickformat='.0%')
#     fig.add_shape(row = 1, col = i, type="line", x0=0, x1=0, y0=-1, y1=20, yref='paper',line=dict(color="Black", width=1, dash="solid"))

# fig.update_yaxes(row = 1, col = 1, title_font = {"size": 12}, tickfont = dict(size = 11))

# fig.show()

## Código para exportar html
> Apenas markdown e outputs